In [1]:
# Delete this cell to re-enable tracebacks
import sys
ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                   exception_only=False, running_compiled_code=False):
    etype, value, tb = sys.exc_info()
    return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

In [2]:
# JSON output syntax highlighting
from __future__ import print_function
from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import HtmlFormatter
from IPython.display import HTML

original_print = print

def json_print(inpt):
    string = str(inpt)
    if string[0] == '{':
        formatter = HtmlFormatter()
        return HTML('<style type="text/css">{}</style>{}'.format(
                    formatter.get_style_defs('.highlight'),
                    highlight(string, JsonLexer(), formatter)))
    else:
        original_print(inpt)

print = json_print

## Creating STIX Content

### Creating STIX Domain Objects

To create a STIX object, provide keyword arguments to the type's constructor:

In [3]:
from stix2 import Indicator

indicator = Indicator(name="File hash for malware variant",
                      labels=["malicious-activity"],
                      pattern="[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']")
print(indicator)

Certain required attributes of all objects will be set automatically if not provided as keyword arguments:

-  If not provided, ``type`` will be set automatically to the correct type. You can also provide the type explicitly, but this is not necessary:

In [4]:
indicator2 = Indicator(type='indicator',
                       labels=["malicious-activity"],
                       pattern="[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']")

Passing a value for ``type`` that does not match the class being constructed will cause an error:

In [5]:
indicator3 = Indicator(type='xxx',
                       labels=["malicious-activity"],
                       pattern="[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']")

InvalidValueError: Invalid value for Indicator 'type': must equal 'indicator'.

-  If not provided, ``id`` will be generated randomly. If you provide an
   ``id`` argument, it must begin with the correct prefix:

In [6]:
indicator4 = Indicator(id="campaign--63ce9068-b5ab-47fa-a2cf-a602ea01f21a",
                       labels=["malicious-activity"],
                       pattern="[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']")

InvalidValueError: Invalid value for Indicator 'id': must start with 'indicator--'.

For indicators, ``labels`` and ``pattern`` are required and cannot be set automatically. Trying to create an indicator that is missing one of these properties will result in an error:

In [7]:
indicator = Indicator()

MissingPropertiesError: No values for required properties for Indicator: (labels, pattern).

However, the required ``valid_from`` attribute on Indicators will be set to the current time if not provided as a keyword argument.

Once created, the object acts like a frozen dictionary. Properties can be accessed using the standard Python dictionary syntax:

In [8]:
indicator['name']

u'File hash for malware variant'

Or access properties using the standard Python attribute syntax:

In [9]:
indicator.name

u'File hash for malware variant'

Attempting to modify any attributes will raise an error:

In [10]:
indicator['name'] = "This is a revised name"

TypeError: 'Indicator' object does not support item assignment

In [11]:
indicator.name = "This is a revised name"

ImmutableError: Cannot modify 'name' property in 'Indicator' after creation.

To update the properties of an object, see the **Versioning** section.

Creating a Malware object follows the same pattern:

In [12]:
from stix2 import Malware

malware = Malware(name="Poison Ivy",
                  labels=['remote-access-trojan'])
print(malware)

As with indicators, the ``type``, ``id``, ``created``, and ``modified`` properties will be set automatically if not provided. For Malware objects, the ``labels`` and ``name`` properties must be provided.

### Creating Relationships

STIX 2 Relationships are separate objects, not properties of the object on either side of the relationship. They are constructed similarly to other STIX objects. The ``type``, ``id``, ``created``, and ``modified`` properties are added automatically if not provided. Callers must provide the ``relationship_type``, ``source_ref``, and ``target_ref`` properties.

In [13]:
from stix2 import Relationship

relationship = Relationship(relationship_type='indicates',
                            source_ref=indicator.id,
                            target_ref=malware.id)
print(relationship)

The ``source_ref`` and ``target_ref`` properties can be either the ID's of other STIX objects, or the STIX objects themselves. For readability, Relationship objects can also be constructed with the ``source_ref``, ``relationship_type``, and ``target_ref`` as positional (non-keyword) arguments:

In [14]:
relationship2 = Relationship(indicator, 'indicates', malware)
print(relationship2)

### Creating Bundles

STIX Bundles can be created by passing objects as arguments to the Bundle constructor. All required properties (``type``, ``id``, and ``spec_version``) will be set automatically if not provided, or can be provided as keyword arguments:

In [15]:
from stix2 import Bundle

bundle = Bundle(indicator, malware, relationship)
print(bundle)